In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import rnn_cell_impl as core_rnn_cell_impl

In [2]:
batch_size = 32
input_size = 1
max_len = 10
rnn_size = 32

In [3]:
cell = tf.contrib.rnn.GRUCell(rnn_size)
encoder_inputs= []    
decoder_inputs = []
for i in range(max_len):
    encoder_inputs.append(tf.placeholder(
        tf.float32, [batch_size, input_size], name="EncoderInput%d" % i))
for i in range(max_len + 1):
    decoder_inputs.append(tf.placeholder(
        tf.float32, [batch_size, input_size], name="DecoderInput%d" % i))
encoder_inputs, decoder_inputs 

([<tf.Tensor 'EncoderInput0:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput1:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput2:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput3:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput4:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput5:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput6:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput7:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput8:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'EncoderInput9:0' shape=(32, 1) dtype=float32>],
 [<tf.Tensor 'DecoderInput0:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'DecoderInput1:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'DecoderInput2:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'DecoderInput3:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'DecoderInput4:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'DecoderInput5:0' shape=(32, 1) dtype=float32>,
  <tf.Tensor 'DecoderIn

In [4]:
# encoder_inputs_once = tf.placeholder(tf.float32,[32,1,10])
encoder_outputs, final_state = tf.contrib.rnn.static_rnn(cell,encoder_inputs, dtype=tf.float32)
# encoder_outputs, final_state = tf.nn.dynamic_rnn(cell,encoder_inputs_once, dtype=tf.float32)
encoder_outputs

[<tf.Tensor 'rnn/rnn/gru_cell/add:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_1:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_2:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_3:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_4:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_5:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_6:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_7:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_8:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_9:0' shape=(32, 32) dtype=float32>]

In [5]:
# print(encoder_outputs)
# print(np.asarray([tf.zeros([32,32])]).shape)
encoder_outputs = [tf.zeros([batch_size, 32])] + encoder_outputs
encoder_outputs

[<tf.Tensor 'zeros:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_1:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_2:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_3:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_4:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_5:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_6:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_7:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_8:0' shape=(32, 32) dtype=float32>,
 <tf.Tensor 'rnn/rnn/gru_cell/add_9:0' shape=(32, 32) dtype=float32>]

In [6]:
top_states = [tf.reshape(e, [-1, 1, cell.output_size]) for e in encoder_outputs]
attention_states = tf.concat(axis=1, values=top_states)
print(attention_states)

Tensor("concat:0", shape=(32, 11, 32), dtype=float32)


encoder를 만든다. (batch_size , sequence , input dim) 더미 아웃풋을 위해 시퀀스를 +1해준다

(원래는 10개)

pointer decoder에 해당하는 값을 decoder_inputs값과 함께 넣어준다.

In [9]:
def pointer_decoder(decoder_inputs, final_state, attention_states, cell, feed_prev=True, dtype=tf.float32):
    with tf.variable_scope("point_decoder", reuse=True):
        batch_size = tf.shape(decoder_inputs[0])[0]  #배치 사이즈
        input_size_ = decoder_inputs[0].get_shape()[1].value # input size 1
        attn_length = attention_states.get_shape()[1].value # sequence 11,
        attn_size = attention_states.get_shape()[2].value #input_dim 32
        attention_vec_size = attn_size  # Size of query vectors for attention. 32 

        hidden = tf.reshape(attention_states, [-1, attn_length, 1, attn_size])
        
        k = tf.get_variable(name="AttnW", shape =[1, 1, attn_size, attention_vec_size])
        hidden_features = nn_ops.conv2d(hidden, k, [1, 1, 1, 1], "SAME")  # 32 11 1 32
        v = tf.get_variable(name="AttnV", shape=[attention_vec_size])
        states = [final_state]   # [32 32]
        
        outputs = []
        prev = None
        batch_attn_size = tf.stack([batch_size, attn_size])    # tf.stack([32, 32])  = 64
        attns = tf.zeros(batch_attn_size, dtype=dtype)
        
        print('batch_size',batch_size, 'attn_size',attn_size)
        attns.set_shape([None, attn_size])
        print(attns)
        inps = []
        
        def attention(query):
            """Point on hidden using hidden_features and query."""
            with tf.variable_scope("Attention"):
                y = core_rnn_cell_impl._linear(query, attention_vec_size, True)
                y = array_ops.reshape(y, [-1, 1, 1, attention_vec_size])
                # Attention mask is a softmax of v^T * tanh(...).
                s = math_ops.reduce_sum(
                    v * math_ops.tanh(hidden_features + y), [2, 3])
                return s
            
        print('decoder_inputs length' , len(decoder_inputs))
        for i in range(len(decoder_inputs)):
            if i > 0:
#                 tf.get_variable_scope().reuse_variables()
                print(tf.get_variable_scope())
            inp = decoder_inputs[i]

            if feed_prev and i > 0:
                inp = tf.stack(decoder_inputs)
                inp = tf.transpose(inp, perm=[1, 0, 2])
                inp = tf.reshape(inp, [-1, attn_length, input_size_])
                inp = tf.reduce_sum(inp * tf.reshape(tf.nn.softmax(output), [-1, attn_length, 1]), 1)
                inp = tf.stop_gradient(inp)
                inps.append(inp)

            # Use the same inputs in inference, order internaly

            # Merge input and previous attentions into one vector of the right size.
            x = core_rnn_cell_impl._linear([inp, attns], output_size=cell.output_size, bias=True)
            print('inp {}'.format(inp))
            print('attns {}'.format(attns))
            print('states[-1] {}'.format(states[-1]))
            # Run the RNN.
            cell_output, new_state = cell(x, states[-1])
            states.append(new_state)
            # Run the attention mechanism.
            output = attention(new_state)

            outputs.append(output)
    return outputs, states, inps
    
pointer_decoder(decoder_inputs, final_state, attention_states, cell)
    

batch_size Tensor("point_decoder_2/strided_slice:0", shape=(), dtype=int32) attn_size 32
Tensor("point_decoder_2/zeros:0", shape=(?, 32), dtype=float32)
decoder_inputs length 11
inp Tensor("DecoderInput0:0", shape=(32, 1), dtype=float32)
attns Tensor("point_decoder_2/zeros:0", shape=(?, 32), dtype=float32)
states[-1] Tensor("rnn/rnn/gru_cell/add_9:0", shape=(32, 32), dtype=float32)


ValueError: Trying to share variable rnn/gru_cell/gates/kernel, but specified shape (64, 64) and found shape (33, 64).